In [1]:
import os, re, shutil, javalang
import pandas as pd
## https://www.cnblogs.com/lemonbit/p/6810972.html
VERSION_NUMBER_CODE_FORMAT = "R3_0_1"
VERSION_NUMBER_NATURAL_FORMAT = "3.0.1"
VERSION_PATH = r"eclipse_plugin_3.0.1"
## metrics:

mt_field = pd.read_csv(f"{VERSION_PATH}/plugin_field.csv")
mt_method = pd.read_csv(f"{VERSION_PATH}/plugin_method.csv")
mt_file = pd.read_csv(f"{VERSION_PATH}/plugin_count_interface_class.csv")

_ = pd.merge(mt_field,mt_method,on=['file'],how= "inner")
mt = pd.merge(_,mt_file,on=['file'],how= "inner")
mt.rename(columns={"num_of_interface": "NOI", "num_of_class": "NOT", "line_of_code": "TLOC"}, inplace=True)
mt.drop(columns=["num_of_anony"], inplace=True)
mt.head()

,file,NOF_num,NOF_max,NOF_total,NSF_num,NSF_max,NSF_total,NOM_num,NOM_max,NOM_total,NSM_num,NSM_max,NSM_total,NOI,NOT,TLOC
0,plugins\org.apache.lucene\src\org\apache\lucen...,1,0,0,1,2,2,1,0,0,1,3,3,0,1,365
1,plugins\org.apache.lucene\src\org\apache\lucen...,4,41,45,4,2,2,4,33,34,4,1,1,0,4,768
2,plugins\org.apache.lucene\src\org\apache\lucen...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,80
3,plugins\org.apache.lucene\src\org\apache\lucen...,1,11,11,1,8,8,1,52,52,1,1,1,0,1,1683
4,plugins\org.apache.lucene\src\org\apache\lucen...,1,5,5,1,0,0,1,2,2,1,0,0,0,1,192


In [2]:
## pre and post bug:

def count_bug(bug_file, version):
    bug_info =  pd.read_csv(bug_file)
    _ = bug_info[["file", "bugID", "releaseNo"]]
    post_version = _[_["releaseNo"] == version]
    s = post_version["file"].value_counts()
    bug_num = pd.DataFrame(
        {
            "file": s.index,
            "num_of_bug": s.values
        }
    )
    return bug_num

post_bug = count_bug(r"eclipse_post_bug.csv", VERSION_NUMBER_CODE_FORMAT) #
pre_bug = count_bug(r"eclipse_pre_bug.csv", VERSION_NUMBER_CODE_FORMAT) #
post_bug.head()

,file,num_of_bug
0,eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\...,1
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1
2,eclipse\org.eclipse.core.resources\src\org\ecl...,1
3,eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\...,1
4,eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\...,1


### How do we group the file level into package level?
If the package and plugin are the same, the files (.java code files) will be grouped. 

In [3]:
## merge them:
def change_to_package_level(df, group_by_keys):
    """
    Table group, 
    """
    sum_col = [ i for i in list(df.columns) if "max" not in i]
    sum_col.remove("package")
    sum_col.remove("plugin")
    sum_method = ["sum" for i in sum_col]

    max_col = [ i for i in list(df.columns) if "max" in i]
    max_method = ["max" for i in max_col]

    hehe1 = dict(zip(sum_col, sum_method))
    hehe2 = dict(zip(max_col, max_method))
    hehe1.update(hehe2)

    group = df.groupby(group_by_keys)

    new_table = pd.DataFrame(
        group.agg(
            hehe1
        )
    ).reset_index()
    return new_table

def generate_dataset(bugTable, metricsTable, map_rst_path):
    """
    Merge the table, 
    Column map, 
    """
    file_mapping = pd.read_csv(map_rst_path)
    file_mapping["file"] = file_mapping["file"].map(lambda x: x.replace("#", "\\"))
    _ = pd.merge(metricsTable, file_mapping, on=['file'], how= "inner")
    _["file"] = _["file"].map(lambda x: x.replace("#", "\\").replace("plugins", "eclipse"))
    # _.to_csv("heheheh.csv", index=False)
    ## temporary file: 
    _1 = pd.merge(_, bugTable, on=['file'], how="left")
    clean_z = _1["num_of_bug"].fillna(0)
    _1["num_of_bug"] = clean_z
    ## file level
    file_level_table = _1.copy()
    file_level_table.drop(columns=["java", "package"], inplace = True)
    file_level_table["buggy"] = file_level_table["num_of_bug"].map(lambda x: True if x > 0 else False)
    file_level_table["NOF_avg"] = file_level_table["NOF_total"] / file_level_table["NOF_num"]
    file_level_table["NSF_avg"] = file_level_table["NSF_total"] / file_level_table["NSF_num"]
    file_level_table["NOM_avg"] = file_level_table["NOM_total"] / file_level_table["NOM_num"]
    file_level_table["NSM_avg"] = file_level_table["NSM_total"] / file_level_table["NSM_num"]
    ## package level
    package = _1.copy()
    package["plugin"] = package["file"].map(lambda x: x.split(os.sep)[1])
    package.drop(columns=["java", "file"], inplace = True)
    package_level = change_to_package_level(package, ["plugin", "package"])
    package_level["buggy"] = package_level["num_of_bug"].map(lambda x: True if x > 0 else False)
    package_level["NOF_avg"] = package_level["NOF_total"] / package_level["NOF_num"]
    package_level["NSF_avg"] = package_level["NSF_total"] / package_level["NSF_num"]
    package_level["NOM_avg"] = package_level["NOM_total"] / package_level["NOM_num"]
    package_level["NSM_avg"] = package_level["NSM_total"] / package_level["NSM_num"]
#     group = package_level.groupby(["plugin", "package"])
    return file_level_table, package_level

file_post, package_post = generate_dataset(post_bug, mt, f"{VERSION_PATH}/plugin_metrics_file_map.csv")
file_pre, package_pre = generate_dataset(pre_bug, mt, f"{VERSION_PATH}/plugin_metrics_file_map.csv")

file_post.to_csv(f"{VERSION_PATH}/dataset-eclipse_{VERSION_NUMBER_NATURAL_FORMAT}-filelevel-post.csv", index=False)
file_pre.to_csv(f"{VERSION_PATH}/dataset-eclipse_{VERSION_NUMBER_NATURAL_FORMAT}-filelevel-pre.csv", index=False)
package_post.to_csv(f"{VERSION_PATH}/dataset-eclipse_{VERSION_NUMBER_NATURAL_FORMAT}-packagelevel-post.csv", index=False)
package_pre.to_csv(f"{VERSION_PATH}/dataset-eclipse_{VERSION_NUMBER_NATURAL_FORMAT}-packagelevel-pre.csv", index=False)

In [4]:
file_post.head() 

,file,NOF_num,NOF_max,NOF_total,NSF_num,NSF_max,NSF_total,NOM_num,NOM_max,NOM_total,...,NSM_total,NOI,NOT,TLOC,num_of_bug,buggy,NOF_avg,NSF_avg,NOM_avg,NSM_avg
0,eclipse\org.apache.lucene\src\org\apache\lucen...,1,0,0,1,2,2,1,0,0,...,3,0,1,365,0.0,False,0.00,2.0,0.0,3.00
1,eclipse\org.apache.lucene\src\org\apache\lucen...,4,41,45,4,2,2,4,33,34,...,1,0,4,768,0.0,False,11.25,0.5,8.5,0.25
2,eclipse\org.apache.lucene\src\org\apache\lucen...,0,0,0,0,0,0,0,0,0,...,0,1,0,80,0.0,False,NaN,NaN,NaN,NaN
3,eclipse\org.apache.lucene\src\org\apache\lucen...,1,11,11,1,8,8,1,52,52,...,1,0,1,1683,0.0,False,11.00,8.0,52.0,1.00
4,eclipse\org.apache.lucene\src\org\apache\lucen...,1,5,5,1,0,0,1,2,2,...,0,0,1,192,0.0,False,5.00,0.0,2.0,0.00


In [5]:
package_post.head()

,plugin,package,NOF_num,NOF_total,NSF_num,NSF_total,NOM_num,NOM_total,NSM_num,NSM_total,...,num_of_bug,NOF_max,NSF_max,NOM_max,NSM_max,buggy,NOF_avg,NSF_avg,NOM_avg,NSM_avg
0,org.apache.lucene,org.apache.lucene.demo.html,12,85,12,18,12,113,12,8,...,0.0,41,8,52,3,False,7.083333,1.500000,9.416667,0.666667
1,org.eclipse.ant.core,org.eclipse.ant.core,9,44,9,19,9,135,9,3,...,0.0,18,18,78,2,False,4.888889,2.111111,15.000000,0.333333
2,org.eclipse.ant.core,org.eclipse.ant.internal.core,5,11,5,3,5,25,5,1,...,0.0,6,2,13,1,False,2.200000,0.600000,5.000000,0.200000
3,org.eclipse.ant.core,org.eclipse.ant.internal.core.ant,9,38,9,2,9,128,9,1,...,0.0,24,2,58,1,False,4.222222,0.222222,14.222222,0.111111
4,org.eclipse.ant.ui,org.eclipse.ant.internal.ui.antsupport,7,23,7,5,7,74,7,4,...,0.0,22,2,38,2,False,3.285714,0.714286,10.571429,0.571429
